## Check the setup and connect to the database

In [ ]:
%run "../check_setup.ipynb"

In [ ]:
myconn.get_tables(schema='NHTSA')

# Car complaints data

The __National Highway Traffic Safety Administration (NHTSA)__, is part of the U.S. Department of Transportation.
Complaint information entered into NHTSA’s Office of Defects Investigation __vehicle owner's complaint database__ is used with other data sources to identify __safety issues__ that warrant investigation and to determine if a safety-related defect trend exists. Complaint information is also analyzed to monitor existing recalls for proper scope and adequacy. The NHTSA provides a large dataset of complaints related to cars in the US:[https://www.nhtsa.gov/nhtsa-datasets-and-apis#complaints].
For this demo scenario, we've loaded the 2024 complaints data, for detail instructions see appendix.


In [3]:
# Create a HANA dataframe for the complaints data, pre-loaded into SAP HANA Cloud
hdf_complaints=myconn.table('COMPLAINTS', schema='NHTSA')
# hdf_complaints.get_table_structure()

In [ ]:
# Overview the complaints data
import pandas as pd
pd.set_option('max_colwidth', None) 
display(
    hdf_complaints.filter("""PROD_TYPE='V'""") # Vehicle-related complaint
    .select('CMPLID', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'COMPDESC', 'CDESCR')
    .head(1).collect().T
)

In [ ]:
# Let's filter on specific component-groups, for detailed classification analysis
hdf_carcomplaints=(hdf_complaints
    .select('CMPLID', 'MFR_NAME', 'MAKETXT', 'MODELTXT', 'YEARTXT', 'FUEL_TYPE', 'CRASH', 'FIRE', 'STATE', 'CMPL_TYPE',
            'ANTI_BRAKES_YN', 'CRUISE_CONT_YN', 'DRIVE_TRAIN', 'VEHICLES_TOWED_YN', 'CDESCR', 'COMPDESC')
    .filter('''COMPDESC IN ('AIR BAGS','ELECTRICAL SYSTEM', 'SERVICE BRAKES','STEERING')'''))

hdf_carcomplaints.count()

In [ ]:
pd.set_option('max_colwidth', None) 
display(hdf_carcomplaints.head(1).collect().T)

## Text splitting, preparing complaints description text for vectorization

Text embedding models and Large Language Models (LLMs) often have token length limits, hence managing the text length before running it through such models is a frequent preprocessing task. For that purpose, a new text splitting function __hana_ml.text.text_splitter__ is being introduced and explained in more detail in the following blog post: [Text chunking - an exciting new NLP function in SAP HANA Cloud](https://community.sap.com/t5/technology-blogs-by-sap/text-chunking-an-exciting-new-nlp-function-in-sap-hana-cloud/ba-p/13958766).

In [ ]:
# Determining the character length (using SQL length-fct) of a given text. For western languages, character-length / 3 or 4 is giving an approximate token length
# Note, the text analysis function applied above, also determines the token length specifically

hdf_carcomplaints.select('CMPLID', ('LENGTH("CDESCR")', 'LEN_CDESCR')).sort('LEN_CDESCR', desc=True).head(100).collect()

In [ ]:
hdf_carcomplaints.filter("""CMPLID=1918524""").select("CDESCR").collect().T

In [ ]:
# Applying the Text Splitter with recursive-splitting, available with hana-ml 2.23
from hana_ml.text.text_splitter import TextSplitter

splitter = TextSplitter(split_type='recursive', chunk_size=512, overlap=64)

splitted_text = splitter.split_text(hdf_carcomplaints.select('CMPLID', 'CDESCR').head(10), order_status=True)
#print(splitted_text.shape)
display(splitter.statistics_.collect())

display(splitted_text.collect())

## Generating Text Embeddings in SAP HANA Cloud

SAP HANA Cloud introduces availability of a text embedding model, targeted for vectorization of text data already stored within the database. It is specifically useful for vector engine similarity search scenarios or machine learning tasks, implicitly making use text embedding vectors unlocking the semantic understanding of text data for the analysis. A detailed capability introduction can be found in the following blog post [Text Embedding Service in SAP HANA Cloud Predictive Analysis Library (PAL)](https://community.sap.com/t5/enterprise-resource-planning-blogs-by-sap/text-embedding-service-in-sap-hana-cloud-predictive-analysis-library-pal/ba-p/13958864).

Running vectorizations for 122447 recs took 37 minutes (2244 secs) on 4 vCPUs.

For the sake of the exercise let's reduce the number of records by limiting only to cars with hubrid engine.

In [ ]:
print(f"""Number of records selected for further processing: {(hdf_carcomplaints_he := hdf_carcomplaints.filter(''' "FUEL_TYPE"='HE' ''')).count()}""")

In [ ]:
### Generating Text Embeddings in SAP HANA Cloud with the new PAL function, function available with hana-ml 2.23.
from hana_ml.text.pal_embeddings import PALEmbeddings
pe = PALEmbeddings()
textembeddings = pe.fit_transform(hdf_carcomplaints_he, key="CMPLID", target=["CDESCR"], thread_number=10, batch_size=10) #, max_token_num=512
print(f"{textembeddings.count()} records processed in {round(pe.runtime, 3)} sec")

In [ ]:
textembeddings.get_table_structure()

In [ ]:
# Review the generated Text Embeddings, note they are of vector dimensionality 768 for the current HANA embedding model
cmpl_textembeddings=textembeddings.rename_columns({'VECTOR_COL_CDESCR': 'HANACLOUD_TEXT_EMBEDDING'}).to_tail('COMPDESC')
display(cmpl_textembeddings.head(1).collect().T)